# Продвинутое машинное обучение: 
## Домашнее задание 3
Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса.

Расшифруем несколько предложений простым частотным алгоритмом.

In [20]:
import re
from collections import Counter
import random
import math

In [3]:
with open('corpora/WarAndPeace.txt', 'r') as f:
    text_ru = f.read()
    
text_ru[:1000], len(text_ru)

('Annotation\n\n\n\t"Война и мир" – самый известный роман Льва Николаевича Толстого, как никакое другое произведение писателя, отражает глубину его мироощущения и философии.\tЭта книга из разряда вечных, потому что она обо всем – о жизни и смерти, о любви и чести, о мужестве и героизме, о славе и подвиге, о войне и мире.\tПервый том знакомит с высшим обществом России XIX века. Показаны взаимоотношения между родителями и детьми в семье Ростовых, сватовство у Болконских, интриги у Безуховых, вечера в салоне фрейлины А.П.Шерер, балы в Москве и Петербурге.\n\n\n\n\n\n* * *\n\n\n\nЛев Николаевич ТолстойЧАСТЬ ПЕРВАЯ\n\nЧАСТЬ BTОРАЯ\n\nЧАСТЬ ТРЕТЬЯ\n\n\n\n\n\n* * *\n\n\n\n\n\nЛев Николаевич Толстой\n\nВОЙНА И МИР\n\nТом 1\n\n\n\n\nЧАСТЬ ПЕРВАЯ\n\n\n\n\n\nI\n\n\n— Еh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous préviens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toute

In [4]:
with open('corpora/WarAndPeaceEng.txt', 'r') as f:
    text_eng = f.read()
    
text_eng[:1000]

'\ufeffThe Project Gutenberg EBook of War and Peace, by Leo Tolstoy\n\nThis eBook is for the use of anyone anywhere at no cost and with almost\nno restrictions whatsoever.  You may copy it, give it away or re-use it\nunder the terms of the Project Gutenberg License included with this\neBook or online at www.gutenberg.org\n\n\nTitle: War and Peace\n\nAuthor: Leo Tolstoy\n\nTranslators: Louise and Aylmer Maude\n\nPosting Date: January 10, 2009 [EBook #2600]\n\nLast Updated: March 15, 2013\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK WAR AND PEACE ***\n\nAn Anonymous Volunteer, and David Widger\n\n\n\n\n\nWAR AND PEACE\n\nBy Leo Tolstoy/Tolstoi\n\nCONTENTS\n\nBOOK ONE: 1805\n\nCHAPTER I\n\nCHAPTER II\n\nCHAPTER III\n\nCHAPTER IV\n\nCHAPTER V\n\nCHAPTER VI\n\nCHAPTER VII\n\nCHAPTER VIII\n\nCHAPTER IX\n\nCHAPTER X\n\nCHAPTER XI\n\nCHAPTER XII\n\nCHAPTER XIII\n\nCHAPTER XIV\n\nCHAPTER XV\n\nCHAPTER XVI\n\nCHAPTER XVII\n\nCHAPTER XVIII\n\nCHAPTER XIX\n\nCHAPTER XX\n\nCH

In [5]:
def simple_text_preparation(text, regex):
    text = text.lower()
    text = text.replace('.', ' ')
    text = text.strip()
    text = re.sub(regex, '', text)
    return text

text_ru = simple_text_preparation(text_ru, r'[^а-я ]')
text_end = simple_text_preparation(text_eng, r'[^a-z ]')

counter_ru = Counter(text_ru)
counter_ru.most_common(10)

[(' ', 117256),
 ('о', 61282),
 ('а', 45209),
 ('е', 42519),
 ('и', 35838),
 ('н', 35119),
 ('т', 30619),
 ('с', 28128),
 ('л', 27277),
 ('в', 24824)]

In [63]:
def list_shuffle(l):
    l.remove(' ')
    before_ = l.copy()
    random.shuffle(l)
    after_ = l
    return dict(zip(before_, after_))


def text_shuffle(text, shuffle):
    new_text = list(text)
    for i, letter in enumerate(text):
        if letter in shuffle:
            new_text[i] = shuffle[letter]
    return ''.join(new_text)
    
shuffle = list_shuffle(list(counter_ru))

new_text = text_shuffle(text_ru, shuffle)
    
new_text[:10], text_ru[:10]

('йкюьр е пе', 'война и ми')

Так мы производим замену букв в тексте с помощью случайной пересатновки алфавита. Попробуем восстановить текст по такой расстановке.

In [64]:
sample_size = 1500

train_begin = random.randint(0, len(text_ru) - sample_size)
test_begin = random.randint(0, len(text_ru) - sample_size)
train_text = text_ru[train_begin: train_begin + sample_size]
test_text = text_ru[test_begin: test_begin + sample_size]

train_text, test_text

('розного начальства во всем ужасе представилась его вина и позор в том что он оставшись жив потерял два орудия  он так был взволнован что до сей минуты не успел подумать об этом  смех офицеров еще больше сбил его с толку  он стоял перед багратионом с дрожащею нижнею челюстью и едва проговорил не знаю ваше сиятельство людей не было ваше сиятельство  вы бы могли из прикрытия взятьчто прикрытия не было этого не сказал тушин хотя это была сущая правда  он боялся подвести этим другого начальника и молча остановившимися глазами смотрел прямо в лицо багратиону как смотрит сбившийся ученик в глаза экзаменатору молчание было довольно продолжительно  князь багратион видимо не желая быть строгим не находился что сказать остальные не смели вмешаться в разговор  князь андрей исподлобья смотрел на тушина и пальцы его рук нервически двигались  ваше сиятельство  прервал князь андрей молчание своим резким голосом  вы меня изволили послать к батарее капитана тушина  я был там и нашел две трети людей и 

In [65]:
counter_train = Counter(train_text)
shuffle = list_shuffle(list(counter_train))

train_text_shuffled = text_shuffle(train_text, shuffle)
test_text_shuffled = text_shuffle(test_text, shuffle)
train_text_shuffled, test_text_shuffled

('юшкчшош чуфумибьлу лш лбвт ждубв аювзбьулемуби вош лечу е ашкшю л ьшт фьш шч шбьулсеби дел ашьвюэм злу шюжзеэ  шч ьущ цым лклшмчшлуч фьш зш бвп течжьы чв жбавм ашзжтуьи шц гьшт  бтвр шйенвюшл вяв цшмисв бцем вош б ьшмщж  шч бьшэм авювз цуоюуьешчшт б зюшдуявх чедчвх фвмхбьих е взлу аюшошлшюем чв кчух лусв беэьвмибьлш мхзвп чв цымш лусв беэьвмибьлш  лы цы тшоме ек аюещюыьеэ лкэьифьш аюещюыьеэ чв цымш гьшош чв бщукум ьжсеч ршьэ гьш цыму бжяуэ аюулзу  шч цшэмбэ ашзлвбье гьет зюжошош чуфумичещу е тшмфу шбьучшлелсетебэ омукуте бтшьювм аюэтш л менш цуоюуьешчж щущ бтшьюеь бцелсепбэ жфвчещ л омуку гщкутвчуьшюж тшмфучев цымш зшлшмичш аюшзшмдеьвмичш  щчэки цуоюуьешч лезетш чв двмуэ цыьи бьюшоет чв чуршзембэ фьш бщукуьи шбьумичыв чв бтвме лтвсуьибэ л юукошлшю  щчэки учзювп ебашзмшциэ бтшьювм чу ьжсечу е аумины вош южщ чвюлефвбще злеоумеби  лусв беэьвмибьлш  аювюлум щчэки учзювп тшмфучев блшет ювкщет ошмшбшт  лы твчэ еклшмеме ашбмуьи щ цуьуювв щуаеьучу ьжсечу  э цым ьут е чусвм злв ьювье мхзвп е 

In [66]:
counter_decode = Counter(test_text_shuffled)

encoded_freq_order = [letter[0] for letter in sorted(counter_decode.items(), key=lambda x: x[1], reverse=True)]
initial_freq_order = [letter[0] for letter in sorted(counter_ru.items(), key=lambda x: x[1], reverse=True)]

# make encoding
for i in range(max(len(encoded_freq_order) - len(initial_freq_order), 0)):
    initial_freq_order.append('*')
    
decoder = dict(zip(encoded_freq_order, initial_freq_order))

decoded_text = list(test_text_shuffled)
for i, letter in enumerate(decoded_text):
    decoded_text[i] = decoder[letter]

''.join(decoded_text)

' ойимаепге жауи рткыь вчомин бгслдо весено рар от и всеума чомин рар бзмло зпгжнетто своипи лодоянивгпи патедапи ядемславнкк ядоливояонойтосль слдоуопз яодкмрз мопа в ло йе путоветие боньжие шасг ядобини мва и лотрип уоносроп олоыванись в уослитоц мдзуие  рткыь ослатовинск иыяом вискшич узслгч бдовец ойивнеттге бнеслкюие слдоуие унаыа оункмени всеч и ослатовинись та пономоц рткуите  пономак рткуитк исяглгвана в ло вдепк ло шзвслво рарое исяглгвахл ядимводтге та эадсроп вгчоме ло шзвслво слдача и яошлетик ролодое воыбзйман щлол сладир во всеч ядибнийеттгч  от яоунамин рткуитх яо уонове и яолоп теноврип мвийетиеп яолдеяан ее яо ыалгнрз  к дам к дам  ядоуоводин от и ядисланьто еюе выунктзв ец в унаыа бгслдо оложен и сен та свое песло   самилесь самилесь пичаин иватовиш самилесь от зраыан тевеслре песло яомне себк  офиэиатл оломвитзн мнк тее слзн  уо уо  сраыан сладир оункмгвак ее ордзунеттзх ланих   яолодояинась течодожоот ыаспекнск сзчо чономто теядиклто рар от всеума спекнск омтип длоп

 Получается, конечно, не очень. Можно лучше

## Проделаем то же самое для биграмм

In [67]:
bigrams = []
list_text_ru = list(text_ru)
for a, b in zip(list_text_ru[:-1], list_text_ru[1:]):
    if a != ' ' and b != ' ':
        bigrams.append(a + b)
        

In [68]:
bigrams_counter_ru = Counter(bigrams)

bigrams_encoded = []
list_text_ru = list(test_text_shuffled)
for a, b in zip(list_text_ru[:-1], list_text_ru[1:]):
    if a != ' ' and b != ' ':
        bigrams_encoded.append(a + b)
    
bigrams_encoded_counter = Counter(bigrams_encoded)

encoded_freq_order = [letter[0] for letter in sorted(bigrams_encoded_counter.items(), key=lambda x: x[1], reverse=True)]
initial_freq_order = [letter[0] for letter in sorted(bigrams_counter_ru.items(), key=lambda x: x[1], reverse=True)]
    
decoder = dict(zip(encoded_freq_order, initial_freq_order))

decoded_text = list(test_text_shuffled)
for i, letter in enumerate(decoded_text[:-1]):
    if decoded_text[i] != ' ' and decoded_text[i+1] != ' ':
        decoded_text[i] = decoder[decoded_text[i] + decoded_text[i+1]][0]

''.join(decoded_text)

' дииожисев тяое ентки ьпкиам кзтенш етоблш нвщ рч е аоиыоу пкиам нвщ узегш цсыылзнош акемие глнхбсмырие мсоддрие овшутоиэмиэ онесмкхрвлдчовтии тенлодаж рлятеуж одму л гш дв геооезрав евмбжав шеьы онаеасе ату е грншмт овлваодт еглутпсчеи л овтсрогп увжоав  ентки втосооааусэ усркз ачсаатр хатвнр еноееп димэлзннев олттевоав тенлоав тсрну лтмтубсе аокр е втосооаасчеи оу свлкогп енжоров  свлкоиэ енжорнэ чввувктпсу л гш ьвидэ гш ллатбкш нвосв чввувктеыь оаиаклтнев оу оияаодт ынпкув гш ллатбкш тедьсу е рягтзраэ оеглнсв клуущаопм ыгеь тоиалщ кш аокр оауосидзнннр  рч рлтслиам енжорлх рш овлоев е регдт облогшмт ааидзраит реевчтпм ав рш нсврвуж  э длз э длз  онлооклаам рч е оачтопмоош пов лгтмаплл еп л тсрну кзтенш егбибм е обм оу аксв чттгш   илинттеи илинттеи итсыам мтсооааф илинттеи рч бнрнпм оуеттзпв чттгш ркблв осбэ  оааессяь егкаарпем бмэ оав тпем  ош ош  анрнпм тоиалщ лтмтсктиэ ав нувжтлзнпзх опспх   реглнхиасееи окплнбазрч неичмсусэ дйпш пвлкдош очоаащоош нвщ рч аоиыоу ичмсусэ кдрмт егдт

# MCMC

Для каждого шифра (правила замены символа) однозначно определяется ключ. Наша задача найти ключ, имея информацию о том, с какой частотой распределены биграммы в языке. Мы будем сюмплировать ключи и двигаясь в направлении наибольшего правдоподобия расшифрованного текста, сойдемся к стационарному распределению. В качестве меры качества расшифровки будем считать 

$score = \sum_{i=1}^{n} f_i \log{p_i},$

где $p_i$ - частота i-ой биграммы в Войне и Мире, а $f_i$ - в расшифрованном тесте. Чем точнее расшифровываем, тем выше score. 

In [95]:
# calculate score

def decoding_score(text):
    decoded_freq = {}
    list_text = list(text)
    for a, b in zip(list_text[:-1], list_text[1:]):
        if a != ' ' and b != ' ':  # count occurence of every bigram in the text
            bigram = a + b
            if bigram in decoded_freq:
                decoded_freq[bigram] += 1
            else:
                decoded_freq[bigram] = 1
    score = 0
    for k, v in decoded_freq.items():
        if k in bigrams_counter_ru:
            score += v * math.log(bigrams_counter_ru[k]) 
        
    return score

t = "дви оэ къисзжу к сжэюпои поунжхг лжсэолоаэчр коймб нломплжэмпко эжроаюеиимю сж аиликэит нлжшох  япо пжбои япо пжбои но бох мплиуютп бпо мплиуюип  мнлжвгкжу ломпок локэююмд м лцммбгхг г жкмплгймбгхг моуажпжхг ыизжквгхг нилихивжээчхг поунжхг эжнилилис иьо аолоьг  ж яолп гр сэжип кмир ноыгу нлонжажй км  опкияжуг ихц нолцммбг ноэихишбг г нояивмбг поунч ыиьцегр г эиноэгхжквгр пояэо пжб зи бжб г оэ поьо япо пцп аиужуомд  ыий эихшик  блгяжу оагэ  ж яолп гр аилг  гсхиээгбок      б яолпц щпгр лцммбгр  япопо коляжу эихиш эимбоудбо лжэиэчр вуг но аолоьи  лцьжпиудмпкж блгбг мпоэч мугкжугмд к оагэ оыегй ьцу  мплиудыж сжпгруж г бжб нопох цсэжу ломпок мплиуюуг алць к алцьж лцммбги г жкмплгймбги моуажпч ыози хой япо з щпо пжбои  ацхжу ломпок   г саимд ьаи кмюбцт хгэцпц ьомцажлд хозип цкгажпд гр эо эип щпо килэо поудбо эимбоудбо хилсжкшик  щпо нлойаип щпо эи по щпо эи хозип ычпд  ацхжу оэ   поудбо номболии номболии нлоиржпд грхчмуд о нолжзиэгг г ыиьмпки эи хоьуж нлгппг к ьоуокц ломпокц  ропю оэ г кгаиу"
decoding_score(t)

2639.108615172398

В качестве ключа будем использовать последовательность букв, которые и будут задавать соответствие расшифровки

In [70]:
def swap_letters(sh):
    """Это функция, которая меняет в ключе два символа между собой."""
    shuffle = sh.copy()
    pos_1, pos_2 = random.choices(list(shuffle.keys()), k=2)
    if pos_1 == pos_2:
        return swap_letters(shuffle)
    value_1 = shuffle[pos_1]
    value_2 = shuffle[pos_2]
    shuffle[pos_2] = value_1
    shuffle[pos_1]= value_2
    return shuffle


In [83]:
def MCMC_decrypt(n_iter, encoded_text, sh):
    shuffle = sh.copy()
    best_key = {}
    score = 0
    print(shuffle, encoded_text)
    for i in range(n_iter):
        # изменим две буквы в текущем ключе
        new_shuffle = swap_letters(shuffle)
        
        # расшифруем закодированный текст с помощью двух вариантов ключа
        new_decoded_text = text_shuffle(encoded_text, new_shuffle)
        decoded_text = text_shuffle(encoded_text, shuffle)
        
        # посчитаем score для двух вариантов ключей
        current_score = decoding_score(decoded_text)
        new_score = decoding_score(new_decoded_text)
        
        acceptance_probability = min(1, math.exp(new_score - current_score))
        if current_score > score:
            best_key = shuffle
            score = current_score
        if random.uniform(0, 1) < acceptance_probability:
            shuffle = new_shuffle
        if i%500==0:
            print(f"{i=} {decoded_text[0:99]}")
    return best_key, decoded_text


random_shuffle = list_shuffle(list(counter_ru))
MCMC_decrypt(10000, train_text_shuffled, random_shuffle)


{'в': 'х', 'о': 'ю', 'й': 'ф', 'н': 'ц', 'а': 'о', 'и': 'ъ', 'м': 'с', 'р': 'к', 'с': 'в', 'ы': 'ь', 'з': 'ш', 'е': 'я', 'т': 'з', 'л': 'й', 'ь': 'э', 'к': 'г', 'ч': 'ы', 'г': 'р', 'д': 'п', 'у': 'д', 'п': 'е', 'я': 'ч', 'ж': 'у', 'б': 'и', 'щ': 'а', 'ф': 'л', 'э': 'м', 'х': 'т', 'ю': 'ж', 'ш': 'н', 'ц': 'щ', 'ъ': 'б'} юшкчшош чуфумибьлу лш лбвт ждубв аювзбьулемуби вош лечу е ашкшю л ьшт фьш шч шбьулсеби дел ашьвюэм злу шюжзеэ  шч ьущ цым лклшмчшлуч фьш зш бвп течжьы чв жбавм ашзжтуьи шц гьшт  бтвр шйенвюшл вяв цшмисв бцем вош б ьшмщж  шч бьшэм авювз цуоюуьешчшт б зюшдуявх чедчвх фвмхбьих е взлу аюшошлшюем чв кчух лусв беэьвмибьлш мхзвп чв цымш лусв беэьвмибьлш  лы цы тшоме ек аюещюыьеэ лкэьифьш аюещюыьеэ чв цымш гьшош чв бщукум ьжсеч ршьэ гьш цыму бжяуэ аюулзу  шч цшэмбэ ашзлвбье гьет зюжошош чуфумичещу е тшмфу шбьучшлелсетебэ омукуте бтшьювм аюэтш л менш цуоюуьешчж щущ бтшьюеь бцелсепбэ жфвчещ л омуку гщкутвчуьшюж тшмфучев цымш зшлшмичш аюшзшмдеьвмичш  щчэки цуоюуьешч лезетш чв двмуэ

({'в': 'л',
  'о': 'ч',
  'й': 'э',
  'н': 'щ',
  'а': 'ш',
  'и': 'в',
  'м': 'а',
  'р': 'ж',
  'с': 'ы',
  'ы': 'б',
  'з': 'г',
  'е': 'т',
  'т': 'я',
  'л': 'д',
  'ь': 'о',
  'к': 'у',
  'ч': 'е',
  'г': 'ц',
  'д': 'й',
  'у': 'р',
  'п': 'ь',
  'я': 'ф',
  'ж': 'з',
  'б': 'с',
  'щ': 'к',
  'ф': 'п',
  'э': 'м',
  'х': 'х',
  'ю': 'и',
  'ш': 'н',
  'ц': 'ю',
  'ъ': 'ъ'},
 'инуенчн ерправсодр дн дсля зйрсл шилгсордтарсв лчн дтер т шнуни д оня пон не нсордытсв йтд шнолима гдр низгтм  не орк юба дуднаендре пон гн сль ятезоб ел зсшла шнгзяров ню цоня  сялж нэтщлинд лфл юнавыл сюта лчн с онакз  не сонма шлилг юрчиротненя с гинйрфлх етйелх плахсовх т лгдр шинчнднита ел уерх дрыл стмолавсодн ахгль ел юбан дрыл стмолавсодн  дб юб янчат ту шиткиботм думовпон шиткиботм ел юбан цончн ел скрура озыте жном цон юбар сзфрм ширдгр  не юнмасм шнгдлсот цотя гизчнчн ерправеткр т янапр нсорендтдытятсм чарурят сяноила шимян д атщн юрчиротнез крк сяноито сютдытьсм зплетк д чарур цкурялерониз янап

Таким образом алгоритм MCMC сошелся за 1000 итераций и полностью расшифровал текст.

### Расшифровка текста

In [84]:
main_text = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

main_text

'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [80]:
geo_counter

Counter({'დ': 22,
         'ჳ': 10,
         'ჵ': 13,
         'Ⴢ': 15,
         'Ⴈ': 34,
         'შ': 7,
         'Ⴜ': 6,
         'ხ': 4,
         'ჲ': 12,
         'ს': 13,
         'Ⴙ': 22,
         'Ⴍ': 8,
         'ჾ': 4,
         'Ⴃ': 13,
         'ი': 5,
         'ჰ': 4,
         'Ⴒ': 5,
         'Ⴇ': 6,
         'Ⴉ': 7,
         'Ⴧ': 3,
         'Ⴀ': 1,
         'ქ': 4,
         'ჱ': 3,
         'ჶ': 2,
         'Ⴝ': 3,
         'ე': 2,
         'Ⴞ': 1,
         'ნ': 1})

In [88]:
# подготовим случайный ключ
geo_counter = Counter(main_text)
len(geo_counter) - len(counter_ru)

russian_letters = list(counter_ru)
georgian_letters = list(geo_counter)

# дополним грузинские буквы буквами латинского алфавита, чтобы перепробывать все буквы
new_letters = list('qwert')
georgian_letters = georgian_letters + new_letters

random.shuffle(georgian_letters)

dict(zip(georgian_letters, russian_letters))

{'შ': 'в',
 'ჰ': 'о',
 'ჶ': 'й',
 'ჱ': 'н',
 'r': 'а',
 'ჲ': ' ',
 'ე': 'и',
 'ჳ': 'м',
 'Ⴢ': 'р',
 'ხ': 'с',
 'დ': 'ы',
 'e': 'з',
 'Ⴧ': 'е',
 'ს': 'т',
 'Ⴇ': 'л',
 'ნ': 'ь',
 'ჵ': 'к',
 't': 'ч',
 'Ⴜ': 'г',
 'w': 'д',
 'ჾ': 'у',
 'Ⴝ': 'п',
 'Ⴍ': 'я',
 'Ⴒ': 'ж',
 'Ⴃ': 'б',
 'ქ': 'щ',
 'Ⴙ': 'ф',
 'Ⴉ': 'э',
 'Ⴈ': 'х',
 'q': 'ю',
 'Ⴞ': 'ш',
 'Ⴀ': 'ц',
 'ი': 'ъ'}

In [89]:
random_shuffle = dict(zip(georgian_letters, russian_letters))

MCMC_decrypt(10000, main_text, random_shuffle)


{'შ': 'в', 'ჰ': 'о', 'ჶ': 'й', 'ჱ': 'н', 'r': 'а', 'ჲ': ' ', 'ე': 'и', 'ჳ': 'м', 'Ⴢ': 'р', 'ხ': 'с', 'დ': 'ы', 'e': 'з', 'Ⴧ': 'е', 'ს': 'т', 'Ⴇ': 'л', 'ნ': 'ь', 'ჵ': 'к', 't': 'ч', 'Ⴜ': 'г', 'w': 'д', 'ჾ': 'у', 'Ⴝ': 'п', 'Ⴍ': 'я', 'Ⴒ': 'ж', 'Ⴃ': 'б', 'ქ': 'щ', 'Ⴙ': 'ф', 'Ⴉ': 'э', 'Ⴈ': 'х', 'q': 'ю', 'Ⴞ': 'ш', 'Ⴀ': 'ц', 'ი': 'ъ'} დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ
i=0 ымкрхвгхврср ыхтфяубкътгохркрхжфл рхтфяубкътгох ыэм хехц фщфхмффнйытрпхэф фягохкыщэфхжяфлр б ъхмэфя
i=500 огмнизьизнюндоитавкрметьчинмнихасднитавкрметьчидолгдияийдажаигаафпотншиладавьчиможлаихвасндрдеиглав
i=1000 огинемыемнюндоетавкристыченинежазднетавкристычедолгдеяейдабаегаахпотнцеладавычеиоблаежвазндрдсеглав
i=1500 окинемуемнчндоетавыристуженинегазднетавыристужедолкдеяещдабаекаахпотнцеладавужеиоблаегвазндрдсеклав
i=2000 акинему

({'შ': 'к',
  'ჰ': 'ж',
  'ჶ': 'ш',
  'ჱ': 'б',
  'r': ' ',
  'ჲ': 'д',
  'ე': 'й',
  'ჳ': 'р',
  'Ⴢ': 'н',
  'ხ': 'с',
  'დ': 'а',
  'e': 'ф',
  'Ⴧ': 'я',
  'ს': 'т',
  'Ⴇ': 'з',
  'ნ': 'ь',
  'ჵ': 'и',
  't': 'щ',
  'Ⴜ': 'у',
  'w': 'ъ',
  'ჾ': 'ы',
  'Ⴝ': 'ц',
  'Ⴍ': 'в',
  'Ⴒ': 'г',
  'Ⴃ': 'л',
  'ქ': 'м',
  'Ⴙ': 'о',
  'Ⴉ': 'п',
  'Ⴈ': 'е',
  'q': 'э',
  'Ⴞ': 'х',
  'Ⴀ': 'ю',
  'ი': 'ч'},
 'аринекуекнсндаетовыличтуженинегозднетовыличтужедапрдеяеюдомоерообшатнцеподовужеиампоегвозндлдчерповааекрамоекуекраерсаилинегвлкничтоенегоиязндаеылпрныличтужеблииейлегориастааезадкавдоаейлслтнаепяврлеходцепотазтоецетнзамоетаеобашль')

Не хватает пробела. Сейчас мы его добавим в биграммы несколькими способами.

Способ 1

In [111]:
main_text = main_text.replace('Ⴈ', ' ')  # просто из текста очевидно, какой символ обозначает пробел
main_text

'დჳჵჂ შႼ შჂხჂჲდ სႹႭჾႣჵისႼჰ ჂჵჂ ႲႹႧჲჂ სႹႭჾႣჵისႼჰ ჲდႩჳჲ Ⴧ ႠჲႹქႹ ჳႹႹჱჶდსჂႽ ႩႹჲႹႭႼჰ ჵდქႩႹ ႲႭႹႧჂჲႣჲი ჳႩႹႭდდ შჳდქႹ შႼ შჳდ ჳხდჵႣჵჂ ႲႭႣშჂჵისႹ Ⴢ ႲႹჵჇႧჂჲდ ჾႣႩჳჂჾႣჵისႼჰ ჱႣჵჵ ეႣ ႲႹჳჵდხსდდ ႧდჲშდႭჲႹდ ეႣხႣსჂდ ႩჇႭჳႣ ႾႹჲႽ ႩႹსდႧსႹ Ⴝ სჂႧდქႹ სდ ႹჱდჶႣნ'

In [112]:
geo_counter = Counter(main_text)

russian_letters = list(counter_ru)
georgian_letters = list(geo_counter)


# дополним грузинские буквы буквами латинского алфавита, чтобы перепробывать все буквы
new_letters = list('qwert')
georgian_letters = georgian_letters + new_letters

random.shuffle(georgian_letters)

random_shuffle = dict(zip(georgian_letters, russian_letters))

MCMC_decrypt(40000, main_text, random_shuffle)

{'Ⴇ': 'в', 'ნ': 'о', 't': 'й', 'Ⴉ': 'н', 'დ': 'а', 'w': ' ', 'შ': 'и', 'r': 'м', 'q': 'р', 'ქ': 'с', 'Ⴈ': 'ы', 'Ⴜ': 'з', 'Ⴢ': 'е', 'ჵ': 'т', 'Ⴞ': 'л', 'Ⴒ': 'ь', 'ჾ': 'к', 'Ⴃ': 'ч', 'ს': 'г', 'Ⴍ': 'д', 'ი': 'у', 'ჶ': 'п', 'Ⴝ': 'я', 'Ⴀ': 'ж', 'ჰ': 'б', 'ხ': 'щ', 'ჲ': 'ф', 'Ⴙ': 'э', 'Ⴧ': 'х', 'ჱ': 'ю', 'ე': 'ш', 'ჳ': 'ц', 'e': 'ъ'} დჳჵჂ შႼ შჂხჂჲდ სႹႭჾႣჵისႼჰ ჂჵჂ ႲႹႧჲჂ სႹႭჾႣჵისႼჰ ჲდႩჳჲ Ⴧ ႠჲႹქႹ ჳႹႹჱჶდსჂႽ ႩႹჲႹႭႼჰ ჵდქႩႹ ႲႭႹႧჂჲႣჲი ჳႩႹႭდდ შჳდქႹ შႼ შჳდ ჳხდჵႣჵჂ ႲႭႣშჂჵისႹ Ⴢ ႲႹჵჇႧჂჲდ ჾႣႩჳჂჾႣჵისႼჰ ჱႣჵჵ ეႣ ႲႹჳჵდხსდდ ႧდჲშდႭჲႹდ ეႣხႣსჂდ ႩჇႭჳႣ ႾႹჲႽ ႩႹსდႧსႹ Ⴝ სჂႧდქႹ სდ ႹჱდჶႣნ
i=0 ацте из иещефа гэдкчтугзб ете ьэвфе гэдкчтугзб фанцф х жфэсэ цээюпагея нэфэдзб таснэ ьдэвефчфу цнэд
i=500 евно гс гопоте лаушинылсь оно закто лаушинылсь тедвт ц этара ваажчелоб датаусь нерда зуакотиты вдау
i=1000 ерно вс вомоте лаушинылсь оно закто лаушинылсь тедрт ц этага раапщелоч датаусь негда зуакотиты рдау
i=1500 ерна кс камате ловшинылсь ана зодта ловшинылсь тепрт я этого рообщелач потовсь негпо зводатиты рпов
i=2000 ерна кс

i=36000 ерни пь пидите ловзаныльш ини хости ловзаныльш текрт у йтого рообмелия котовьш негко хвоситаты рков
i=36500 ерни пь пидите ловзанылью ини хости ловзанылью текрт у этого рообмелия котовью негко хвоситаты рков
i=37000 ерни пь пидите ловзаныльш ини хости ловзаныльш текрт у этого рообмелия котовьш негко хвоситаты рков
i=37500 ерни пь пидите ловзаныльш ини хости ловзаныльш текрт у этого рообмелия котовьш негко хвоситаты рков
i=38000 ерни пь пидите ловзаныльш ини хости ловзаныльш текрт у этого рообмелия котовьш негко хвоситаты рков
i=38500 ерни пь пидите ловзаныльш ини хости ловзаныльш текрт у ютого рообмелия котовьш негко хвоситаты рков
i=39000 ерни пь пидите ловзаныльш ини хости ловзаныльш текрт у этого рообмелия котовьш негко хвоситаты рков
i=39500 ерни пь пидите ловзаныльш ини хости ловзаныльш текрт у чтого рообмелия котовьш негко хвоситаты рков


({'Ⴇ': 'с',
  'ნ': 'ю',
  't': 'ъ',
  'Ⴉ': 'к',
  'დ': 'е',
  'w': 'ф',
  'შ': 'п',
  'r': 'щ',
  'q': ' ',
  'ქ': 'г',
  'Ⴈ': 'й',
  'Ⴜ': 'ь',
  'Ⴢ': 'и',
  'ჵ': 'н',
  'Ⴞ': 'б',
  'Ⴒ': 'х',
  'ჾ': 'з',
  'Ⴃ': 'а',
  'ს': 'л',
  'Ⴍ': 'в',
  'ი': 'ы',
  'ჶ': 'д',
  'Ⴝ': 'я',
  'Ⴀ': 'э',
  'ჰ': 'ш',
  'ხ': 'м',
  'ჲ': 'т',
  'Ⴙ': 'о',
  'Ⴧ': 'у',
  'ჱ': 'ж',
  'ე': 'ч',
  'ჳ': 'р',
  'e': 'ц'},
 'ерни пь пидите ловзанылья ини хости ловзанылья текрт у этого рообмелич котовья негко хвоситаты рковее прего пь пре рденани хвапиныло и хонусите закризанылья банн ша хорнедлее сетпевтое шадалие кувра цотч колесло ч лисего ле обемаю')

In [119]:
bigrams = []
list_text_ru = list(text_ru)
for a, b in zip(list_text_ru[:-1], list_text_ru[1:]):
    if a != ' ' and b != ' ':
        bigrams.append(a + b)

bigrams_counter_ru = Counter(bigrams)

In [121]:
russian_letters = list(counter_ru)
georgian_letters = list(geo_counter)

# дополним грузинские буквы буквами латинского алфавита, чтобы перепробывать все буквы
new_letters = list('qwert')
georgian_letters = georgian_letters + new_letters

random.shuffle(georgian_letters)

random_shuffle = dict(zip(georgian_letters, russian_letters))

MCMC_decrypt(100000, main_text, random_shuffle)

{'ჱ': 'в', 'ჰ': 'о', 'Ⴝ': 'й', 'შ': 'н', 'q': 'а', 'ხ': ' ', 'Ⴉ': 'и', 'Ⴇ': 'м', 'w': 'р', 'ჳ': 'с', 'ი': 'ы', 'Ⴈ': 'з', 'Ⴢ': 'е', 'Ⴃ': 'т', 'Ⴀ': 'л', 'ჲ': 'ь', 'Ⴞ': 'к', 'Ⴜ': 'ч', 'ჶ': 'г', 'ს': 'д', 'Ⴧ': 'у', 'Ⴒ': 'п', 't': 'я', 'e': 'ж', 'ნ': 'б', 'Ⴍ': 'щ', 'ქ': 'ф', 'Ⴙ': 'э', 'ე': 'х', 'r': 'ю', 'ჵ': 'ш', 'დ': 'ц', 'ჾ': 'ъ'} დჳჵჂ შႼ შჂხჂჲდ სႹႭჾႣჵისႼჰ ჂჵჂ ႲႹႧჲჂ სႹႭჾႣჵისႼჰ ჲდႩჳჲ Ⴧ ႠჲႹქႹ ჳႹႹჱჶდსჂႽ ႩႹჲႹႭႼჰ ჵდქႩႹ ႲႭႹႧჂჲႣჲი ჳႩႹႭდდ შჳდქႹ შႼ შჳდ ჳხდჵႣჵჂ ႲႭႣშჂჵისႹ Ⴢ ႲႹჵჇႧჂჲდ ჾႣႩჳჂჾႣჵისႼჰ ჱႣჵჵ ეႣ ႲႹჳჵდხსდდ ႧდჲშდႭჲႹდ ეႣხႣსჂდ ႩჇႭჳႣ ႾႹჲႽ ႩႹსდႧსႹ Ⴝ სჂႧდქႹ სდ ႹჱდჶႣნ
i=0 цсше нч не еьц дэщътшыдчо еше пэмье дэщътшыдчо ьцись у льэфэ сээвгцдей иэьэщчо шцфиэ пщэмеьтьы сиэщ
i=500 исте уя уебени ларготыляж ете памне ларготыляж нидсн ю йназа саакцилеч данаряж тизда праменоны сдар
i=1000 ести уя уимине ларготыляж ити пабни ларготыляж недсн ю ьнава саашкелич данаряж тевда прабиноны сдар
i=1500 есви тя тимине ларговыляж иви бакни ларговыляж недсн у ьназа саашпелиц данаряж везда бракиноны сдар
i=2000 етви ся

i=36000 етни ся сичиве лорманыляз ини подви лорманыляз вектв у цвого тообщелиш коворяз негко продивавы ткор
i=36500 етни ся симиве лорчаныляз ини подви лорчаныляз вектв у эвого тообщелиш коворяз негко продивавы ткор
i=37000 етни ся сичиве лорманыляз ини подви лорманыляз вектв у йвого тообщелиш коворяз негко продивавы ткор
i=37500 етни ся сичиве лорманыляз ини подви лорманыляз вектв у ьвого тообщелиш коворяз негко продивавы ткор
i=38000 етна ся сачаве лорминыляж ана подва лорминыляж вектв у звого тообщелаш коворяж негко продавивы ткор
i=38500 етна ся сачаве лорминыляж ана подва лорминыляж вектв у звого тообщелаш коворяж негко продавивы ткор
i=39000 етна ся сачаве лорминыляз ана подва лорминыляз вектв у ьвого тообщелаш коворяз негко продавивы ткор
i=39500 етни ся сичиве лорманыляз ини подви лорманыляз вектв у цвого тообщелиш коворяз негко продивавы ткор
i=40000 етна ся сачаве лоржиныляз ана подва лоржиныляз вектв у цвого тообщелаш коворяз негко продавивы ткор
i=40500 етна ся самаве лоржи

i=75000 етни сь симиве лорханыльш ини подви лорханыльш вектв у явого тоожчелию коворьш негко продивавы ткор
i=75500 етни сь симиве лорханылья ини подви лорханылья вектв у  вого тоожчелиш коворья негко продивавы ткор
i=76000 етни сь симиве лорзанылью ини подви лорзанылью вектв у явого тоожчелиш коворью негко продивавы ткор
i=76500 етни сь симиве лорзанылью ини подви лорзанылью вектв у явого тоожчелиш коворью негко продивавы ткор
i=77000 етни сь симиве лорзанылью ини подви лорзанылью вектв у явого тоожбелиш коворью негко продивавы ткор
i=77500 етни сь симиве лорчанылью ини подви лорчанылью вектв у явого тоозбелиш коворью негко продивавы ткор
i=78000 етни сь симиве лорзанылью ини подви лорзанылью вектв у эвого тообчелиш коворью негко продивавы ткор
i=78500 етни сь симиве лорзанылья ини подви лорзанылья вектв у ъвого тоожчелиш коворья негко продивавы ткор
i=79000 етни сь сичиве лорманылья ини подви лорманылья вектв у ювого тоожбелиш коворья негко продивавы ткор
i=79500 етни сь сичиве лорма

({'ჱ': 'б',
  'ჰ': 'ю',
  'Ⴝ': 'ш',
  'შ': 'с',
  'q': ' ',
  'ხ': 'м',
  'Ⴉ': 'к',
  'Ⴇ': 'д',
  'w': 'ъ',
  'ჳ': 'т',
  'ი': 'ы',
  'Ⴈ': 'ц',
  'Ⴢ': 'и',
  'Ⴃ': 'а',
  'Ⴀ': 'я',
  'ჲ': 'в',
  'Ⴞ': 'х',
  'Ⴜ': 'ь',
  'ჶ': 'ч',
  'ს': 'л',
  'Ⴧ': 'у',
  'Ⴒ': 'п',
  't': 'щ',
  'e': 'ф',
  'ნ': 'й',
  'Ⴍ': 'р',
  'ქ': 'г',
  'Ⴙ': 'о',
  'ე': 'з',
  'r': 'э',
  'ჵ': 'н',
  'დ': 'е',
  'ჾ': 'ж'},
 'есни вь вимите лорзаныльш ини подти лорзаныльш текст у чтого сообщелия которьш негко продитаты скорее всего вь все сменани правиныло и понудите заксизаныльш банн жа поснемлее детвертое жамалие курса хотя коледло я лидего ле обещай')

Становится возможным догадаться, что тут написано. "Если вы видите _ или почти _ текст у этого сообщения, которое легко прочитать, скорее всего вы все сделали правильно и получить максимальный за последнее четвертое задание курса хотя конечно я ничего не обещаю"

### Попробуем то же самое на триграммах

In [132]:
# перепишем функцию подсчета score

def decoding_score(text):
    decoded_freq = {}
    list_text = list(text)
    for a, b, c in zip(list_text[:-2], list_text[1:-1], list_text[2:]):
        if '  ' not in a+b+c:
            trigram = a + b + c
            if trigram in decoded_freq:
                decoded_freq[trigram] += 1
            else:
                decoded_freq[trigram] = 1
    score = 0
    for k, v in decoded_freq.items():
        if k in trigrams_counter_ru:
            score += v * math.log(trigrams_counter_ru[k]) 
        
    return score


trigrams = []
list_text_ru = list(text_ru)
for a, b, c in zip(list_text_ru[:-2], list_text_ru[1:-1], list_text_ru[2:]):
    if '  ' not in a+b+c:
        trigrams.append(a + b + c)

trigrams_counter_ru = Counter(trigrams)

In [142]:
geo_counter = Counter(main_text)
russian_letters = list(counter_ru)
georgian_letters = list(geo_counter)

# дополним грузинские буквы буквами латинского алфавита, чтобы перепробывать все буквы
new_letters = list('qwert')
georgian_letters = georgian_letters + new_letters

russian_letters.remove(' ')
georgian_letters.remove(' ')

random.shuffle(georgian_letters)
random_shuffle = dict(zip(georgian_letters, russian_letters))

MCMC_decrypt(100000, main_text, random_shuffle)

{'w': 'в', 'Ⴙ': 'о', 'Ⴃ': 'й', 'Ⴒ': 'н', 'Ⴀ': 'а', 'ს': 'и', 'ქ': 'м', 'Ⴉ': 'р', 'ჰ': 'с', 'Ⴇ': 'ы', 'e': 'з', 'Ⴝ': 'е', 'q': 'т', 'ჾ': 'л', 'ი': 'ь', 'ჱ': 'к', 'Ⴞ': 'ч', 'r': 'г', 't': 'д', 'ჶ': 'у', 'Ⴍ': 'п', 'ნ': 'я', 'შ': 'ж', 'ჲ': 'б', 'Ⴜ': 'щ', 'Ⴧ': 'ф', 'Ⴢ': 'э', 'ჵ': 'х', 'ხ': 'ю', 'დ': 'ш', 'ჳ': 'ц', 'ე': 'ъ'} დჳჵჂ შႼ შჂხჂჲდ სႹႭჾႣჵისႼჰ ჂჵჂ ႲႹႧჲჂ სႹႭჾႣჵისႼჰ ჲდႩჳჲ Ⴧ ႠჲႹქႹ ჳႹႹჱჶდსჂႽ ႩႹჲႹႭႼჰ ჵდქႩႹ ႲႭႹႧჂჲႣჲი ჳႩႹႭდდ შჳდქႹ შႼ შჳდ ჳხდჵႣჵჂ ႲႭႣშჂჵისႹ Ⴢ ႲႹჵჇႧჂჲდ ჾႣႩჳჂჾႣჵისႼჰ ჱႣჵჵ ეႣ ႲႹჳჵდხსდდ ႧდჲშდႭჲႹდ ეႣხႣსჂდ ႩჇႭჳႣ ႾႹჲႽ ႩႹსდႧსႹ Ⴝ სჂႧდქႹ სდ ႹჱდჶႣნ
i=0 шцхэ жщ жэюэбш иоплйхьищс эхэ ноыбэ иоплйхьищс бшрцб ф абомо цоокушиэе робопщс хшмро нпоыэбйбь цроп
i=500 алне дш дезета ворчинувшь ене посте ворчинувшь таклт ы этого лообщавея которшь нагко просетиту лкор
i=1000 есла дш дачате вормилывшь ала понта вормилывшь текст у этого сообщевая которшь легко пронатиты скор
i=1500 есла чш чадате вормилывшь ала понта вормилывшь текст у этого сообщевая которшь легко пронатиты скор
i=2000 есла чш чадате во

i=35500 если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скор
i=36000 если вы видите нормальный или почти нормальный текст у этого сообжения который легко прочитать скор
i=36500 если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скор
i=37000 если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скор
i=37500 если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скор
i=38000 если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скор
i=38500 если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скор
i=39000 если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скор
i=39500 если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скор
i=40000 если вы видите норма

i=74500 если вы видите норжальным или почти норжальным текст у этого сообщения которым легко прочитать скор
i=75000 если вы видите норжальным или почти норжальным текст у этого сообщения которым легко прочитать скор
i=75500 если вы видите норжальный или почти норжальный текст у этого сообщения который легко прочитать скор
i=76000 если вы видите норжальный или почти норжальный текст у этого сообщения который легко прочитать скор
i=76500 если вы видите норжальный или почти норжальный текст у этого сообщения который легко прочитать скор
i=77000 если вы видите норжальный или почти норжальный текст у этого сообщения который легко прочитать скор
i=77500 если вы видите норжальный или почти норжальный текст у этого сообщения который легко прочитать скор
i=78000 если вы видите норжальный или почти норжальный текст у этого сообщения который легко прочитать скор
i=78500 если вы видите норжальный или почти норжальный текст у этого сообщения который легко прочитать скор
i=79000 если вы видите норжа

({'w': 'ц',
  'Ⴙ': 'о',
  'Ⴃ': 'а',
  'Ⴒ': 'п',
  'Ⴀ': 'э',
  'ს': 'н',
  'ქ': 'г',
  'Ⴉ': 'к',
  'ჰ': 'й',
  'Ⴇ': 'ч',
  'e': 'ф',
  'Ⴝ': 'я',
  'q': 'ъ',
  'ჾ': 'м',
  'ი': 'ь',
  'ჱ': 'б',
  'Ⴞ': 'х',
  'r': 'ж',
  't': 'ш',
  'ჶ': 'щ',
  'Ⴍ': 'р',
  'ნ': 'ю',
  'შ': 'в',
  'ჲ': 'т',
  'Ⴜ': 'ы',
  'Ⴧ': 'у',
  'Ⴢ': 'и',
  'ჵ': 'л',
  'ხ': 'д',
  'დ': 'е',
  'ჳ': 'с',
  'ე': 'з'},
 'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю')

Результаты действительно улучишились после добавления триграмм. Результаты не всегда становятся лучше. тут приходится несколько раз запускать алгоритм, поскольку он сильно зависит от инициализации.